# Object Detection using Ultralytics and 🔭 Galileo

In this tutorial, we'll train a model with the Ultralytics CLI and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

# 1. Install dataquality and ultralytics

In [ ]:
!pip install -U dataquality ultralytics &> /dev/null
print('👋 Installed necessary libraries!')

In [ ]:
#@markdown Check that a GPU is available

import torch
# Check Cuda.
if torch.cuda.is_available():
  print("⚡ You are connected to a GPU!")
else:
  print("❗You are NOT connected to a GPU ❗It is recommended to connect to a GPU before training")

# 2. Download Your Data

Download your data and set it up in YOLO format in this environment. In this example we will use the publicly available SODA datasert hosted remotely in our cloud.

In [ ]:
#@title Download a Dataset hosted by Galileo

dataset_name = "soda" #@param ["soda", "coco-validation"] {allow-input: true}

if dataset_name == "soda":
  archive_name = "soda+bottles.v4-release.yolov8.zip"
elif dataset_name == "coco-validation":
  archive_name = "coco-val2017.zip"

# Create the content folder if we're in CI
!mkdir datasets
!wget https://galileo-s3os-images.s3.amazonaws.com/{archive_name} -O datasets/dataset.zip
!unzip datasets/dataset.zip -d datasets


import os
if os.getenv("MINIMIZE_FOR_CI", "false") == "true":
  # Downsample the dataset (tailored for SODA)
  from pathlib import Path
  import shutil

  train_dir = Path("datasets/train")
  train_dir_images = train_dir / "images"
  train_dir_labels = train_dir / "labels"

  train_dir_tiny = Path("datasets/train_tiny")
  train_dir_images_tiny = train_dir_tiny / "images"
  train_dir_labels_tiny = train_dir_tiny / "labels"
  
  os.makedirs(train_dir_tiny, exist_ok=True)
  os.makedirs(train_dir_images_tiny, exist_ok=True)
  os.makedirs(train_dir_labels_tiny, exist_ok=True)

  image_names = [imm for imm in os.listdir(train_dir_images) if imm.endswith(".jpg")]
  print(f"Found {len(image_names)} image names")
  image_names = image_names[:int(len(image_names) * 0.1)]

  for image_name in image_names:
    shutil.copy(train_dir_images / image_name, train_dir_images_tiny / image_name)
    text_name = image_name.replace('.jpg', '.txt')
    shutil.copy(train_dir_labels / text_name, train_dir_labels_tiny / text_name)

  # Update the Yaml file
  import yaml
  yaml_path = "datasets/data.yaml"
  with open(yaml_path) as f:
      data_yaml = yaml.safe_load(f)

  data_yaml["train"] = data_yaml["train"].replace("train", "train_tiny")
  data_yaml["val"] = data_yaml["train"]
  data_yaml["bucket_val"] = data_yaml["bucket_train"]

  with open(yaml_path, "w") as f:
      yaml.dump(data_yaml, f)


# 3. Set your environment variables and train with Galileo!

In [ ]:
# 🔭🌕 To avoid getting prompted during the execution of dqyolo, set your credentials by uncommenting and populating the below variables.

%env GALILEO_PROJECT_NAME = object-detection-demo
%env GALILEO_RUN_NAME = soda_demo_run
# %env GALILEO_CONSOLE_URL = 
# %env GALILEO_USERNAME = 
# %env GALILEO_PASSWORD = 

In [ ]:
# 🔭🌕 Start training with Galileo!
!dqyolo detect train model=yolov8x data=datasets/data.yaml epochs=3 optimizer=SGD